In [ ]:
!pip install -q segmentation_models_pytorch

In [ ]:
# Install required libraries (if needed)
# !pip install albumentations segmentation_models_pytorch

import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

import segmentation_models_pytorch as smp

In [ ]:
DATA_DIR = '/kaggle/input/tusimple/TUSimple/train_set'
TRAIN_IMAGES_DIR = os.path.join(DATA_DIR, 'clips')
LABELS_FILE = os.path.join(DATA_DIR, 'label_data_0313.json')

print(f"Train Images Path: {TRAIN_IMAGES_DIR}")
print(f"Labels Path: {LABELS_FILE}")

In [ ]:
import json

with open(LABELS_FILE, 'r') as f:
    labels = [json.loads(line) for line in f]

print(labels[0])

In [ ]:
class TuSimpleDataset(Dataset):
    def __init__(self, labels, img_dir, transform=None):
        self.labels = labels
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = self.labels[idx]
        img_path = os.path.join(self.img_dir, item['raw_file'])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Generate binary lane mask (You can improve later)
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
        for lane in item['lanes']:
            for x, y in zip(lane, item['h_samples']):
                if x > 0:
                    cv2.circle(mask, (x, y), 5, 255, -1)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask.unsqueeze(0)  # 1 channel mask

In [ ]:
train_transform = A.Compose([
    A.Resize(360, 640),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(360, 640),
    A.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
    ToTensorV2()
])

In [ ]:
# Split into train/val manually (for now small set)
train_labels = labels[:8000]
val_labels = labels[8000:]

train_dataset = TuSimpleDataset(train_labels, TRAIN_IMAGES_DIR, transform=train_transform)
val_dataset = TuSimpleDataset(val_labels, TRAIN_IMAGES_DIR, transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)

In [ ]:
import segmentation_models_pytorch as smp  # <--- THIS LINE IS IMPORTANT!

model = smp.Unet(
    encoder_name="resnet18",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation=None
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
def train_epoch(model, loader, optimizer, loss_fn, device):
    model.train()
    running_loss = 0
    for imgs, masks in tqdm(loader):
        imgs, masks = imgs.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = loss_fn(outputs, masks)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(loader)

def validate_epoch(model, loader, loss_fn, device):
    model.eval()
    running_loss = 0
    with torch.no_grad():
        for imgs, masks in loader:
            imgs, masks = imgs.to(device), masks.to(device)
            outputs = model(imgs)
            loss = loss_fn(outputs, masks)
            running_loss += loss.item()
    return running_loss / len(loader)

In [ ]:
num_epochs = 10
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer, loss_fn, device)
    val_loss = validate_epoch(model, val_loader, loss_fn, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")